# District Cleaning — Andhra Pradesh

This notebook standardizes **district names** for this state across:
- Enrolment data
- Demographic update data
- Biometric update data

**All data is saved back to the same cleaned files.**

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

CLEAN_DIR = Path("../../data/processed/cleaned")

enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")

for df in [enrol_df, demo_df, bio_df]:
    df["state"] = df["state"].astype(str).str.strip().str.title()
    df["district"] = df["district"].astype(str).str.strip().str.title()

print("✅ All datasets loaded and normalized (Title Case)")


✅ All datasets loaded and normalized (Title Case)


In [2]:
STATE_NAME = "Andhra Pradesh"

districts = sorted(
    set(
        enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
        .tolist()
    )
)

print(f"State: {STATE_NAME}")
print(f"Number of unique districts: {len(districts)}")

pd.DataFrame(
    {"District Name": districts}
)


State: Andhra Pradesh
Number of unique districts: 32


,District Name
0,Alluri Sitharama Raju
1,Anakapalli
2,Anantapur
3,Ananthapur
4,Ananthapuramu
5,Annamayya
6,Bapatla
7,Chittoor
8,Cuddapah
9,Dr. B. R. Ambedkar Konaseema


## District Mapping

Add mappings in **Title Case only**.

Format:
```python
DISTRICT_MAPPING = {
    "Correct District": ["Wrong Name 1", "Wrong Name 2"],
}
```

In [3]:
DISTRICT_MAPPING = {
    # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
    "Ananthapuramu": ["Anantapur", "Ananthapur"],
    "Sri Potti Sriramulu Nellore": ["Nellore", "Spsr Nellore"],
    "NTR": ["Ntr"],
    "Visakhapatnam": ["Visakhapatanam"],
    "Y.S.R. Kadapa": ["Cuddapah", "Y. S. R"]
}

def apply_mapping(df, state, mapping, label):
    total = 0
    for correct, wrongs in mapping.items():
        mask = (
            (df["state"] == state) &
            (df["district"].isin(wrongs))
        )
        count = mask.sum()
        df.loc[mask, "district"] = correct
        total += count
        if count > 0:
            print(f"✔ {label} → {correct} : {count} rows fixed")
    return total

total_fixes = 0
total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")

print(f"✅ Total fixes in {STATE_NAME}: {total_fixes}")


✔ Enrolment → Ananthapuramu : 150 rows fixed
✔ Enrolment → Sri Potti Sriramulu Nellore : 128 rows fixed
✔ Enrolment → Visakhapatnam : 1 rows fixed
✔ Enrolment → Y.S.R. Kadapa : 137 rows fixed
✔ Demographic → Ananthapuramu : 780 rows fixed
✔ Demographic → Sri Potti Sriramulu Nellore : 424 rows fixed
✔ Demographic → Y.S.R. Kadapa : 659 rows fixed
✔ Biometric → Ananthapuramu : 2812 rows fixed
✔ Biometric → Sri Potti Sriramulu Nellore : 1344 rows fixed
✔ Biometric → Y.S.R. Kadapa : 1707 rows fixed
✅ Total fixes in Andhra Pradesh: 8142


In [4]:
enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)

print("💾 All cleaned files saved successfully (overwritten)")


💾 All cleaned files saved successfully (overwritten)
